# openCypher Workshop - Section 05: Advanced openCypher Query Patterns

In this section, we show you some more advanced openCypher queries. There's virtually an infinite number of ways you can chain openCypher clauses and sub-clauses together to build patterns.  In this next notebook, we want to convey some of the more common query patterns and use cases.

## Graph Model

As we have in previous notebooks, we've included here a copy of the graph data model for this dataset as reference to aid in writing your queries.

<img src="https://ee-assets-prod-us-east-1.s3.amazonaws.com/modules/f3f89ef4607743429fb01ae23d983197/v1/workshop/images/datamodel-updated.png"/>


### 05.01 - Recommendation query

One of the more popular use cases for a graph database is for creating recommendations.  You may recall one of our earlier queries that was used to find friends of friends, or co-stars of co-stars.  Building recommendations can be very similar to that.

Start with a person with ID 'person378' and suggest 10 movie recommendations.
The movies should be highly rated and belong to the favorite genre of this person.

In [ ]:
%%oc

MATCH (p)-[:rated]->(m:movie)-[:genre]->(g)
WHERE id(p) = 'person378'
WITH DISTINCT g, count(*) AS frequency, collect(m) AS watched
ORDER BY frequency DESC
LIMIT 1
WITH g AS topgenre, watched
MATCH (topgenre)<-[:genre]-(n:movie)
WHERE n.averageRating > 8.0 AND n.numVotes > 50000 AND NOT (n IN watched)
RETURN n.title, n.averageRating
ORDER BY n.averageRating DESC
LIMIT 10

Let's break this down. First, we'll want to get the favorite genre of person378. We can get this by finding all the movies they rated (i.e. watched), looking at the genre of each of those movies, then seeing which genre of movie they watched the most:

In [ ]:
%%oc

MATCH (p)-[:rated]->(m:movie)-[:genre]->(g)
WHERE id(p) = 'person378'
WITH DISTINCT g, count(*) AS frequency, collect(m) AS watched // count the number of times each genre appeared 
                                                          // also make a collection of the movies they watched 
ORDER BY frequency DESC // order the genres by how frequently they appeared, in descending order
LIMIT 1
RETURN g // return the genre of movie they watch the most.

It looks like person378's most watched genre is drama, so we want to make sure we recommend only movies with a genre of drama. So, we pipe the previous information we found (favorite genre, collection of watched movies) to the next part of the query via `WITH`, to search for all movies of that genre that are highly rated:

In [ ]:
%%oc

MATCH (p)-[:rated]->(m:movie)-[:genre]->(g)
WHERE id(p) = 'person378'
WITH DISTINCT g, count(*) AS frequency, collect(m) AS watched
ORDER BY frequency DESC
LIMIT 1
WITH g AS topgenre, watched // pipe person378's favorite genre and the collection of movies they watched 
                            // to the next part of the query
MATCH (topgenre)<-[:genre]-(n:movie) // gather all the patterns where movies have a genre of drama 
WHERE n.averageRating > 8.0 AND n.numVotes > 50000 AND NOT (n IN watched) // only keep the patterns where 
 // the movie is rated > 8.0, has more than 50,000 votes, and isn't something that person378 has already watched
RETURN n.title, n.averageRating
ORDER BY n.averageRating DESC // sort the remaining patterns from highest movie rating to lowest
LIMIT 10 // recommend the top 10 highest rated drama movies that person378 hasn't watched yet

### 05.02 - The "Six Degrees of Kevin Bacon"

It wouldn't be a complete graph workshop if we didn't include the "Six Degrees of Kevin Bacon"! :)  If you're not familar with the concept, the rule of ["six degrees of separation"](https://en.wikipedia.org/wiki/Six_degrees_of_separation) postulates that everyone on the planet is connected through no more than six social relationships with anyone else on the planet.  That same concept has been extended within the realm of film, suggesting that all actors are no more than six degrees (across common co-stars and productions) connected to the actor Kevin Bacon.  (It is also a humorous joke because it seems like Kevin Bacon has performed in A LOT of movies!)

This next section introduces you to variable-length paths, which we'll use to calculate the number of degrees of separation between artists and Kevin Bacon (AKA the Bacon number). But before we begin, let's try to write some queries without using them.

By definition, Kevin Bacon himself has a Bacon number of 0. So any artists that co-starred in a movie with Kevin Bacon would have a Bacon number of 1. Let's see if Keegan-Michael Key co-starred in a movie with Kevin Bacon:

In [ ]:
%%oc

MATCH p = (a1:Artist)<-[:actor|actress]-(m1:movie)-[:actor|actress]->(bacon:Artist)
WHERE a1.name = 'Keegan-Michael Key' AND bacon.name = 'Kevin Bacon'
RETURN p

We don't get any results, so let's now see if Keegan-Michael Key's co-stars have been co-stars with Kevin Bacon - if so, that would give him a Bacon number of 2:

In [ ]:
%%oc

MATCH p =(a1:Artist)<-[:actor|actress]-(m1:movie)-[:actor|actress]->(a2:Artist)<-[:actor|actress]-(m2:movie)-[:actor|actress]->(bacon:Artist)
WHERE a1.name = 'Keegan-Michael Key' AND bacon.name = 'Kevin Bacon'
RETURN p

We still don't get any results, which means Keegan-Michael Key's Bacon number is greater than 2. We could keep extending the `MATCH` statement above, but there's an easier way to search for the minimum Bacon number, and it's through the use of variable-lenth paths.

Let's first set some variables to make it easier to visualize all the Bacon number paths we will generate:

In [ ]:
node_labels = '{"Artist":"name","movie":"title"}'

Now we can use the following query to find the first 10 shortest Bacon number paths. After running the query, click on the Graph tab to view a visualization of the paths:

In [ ]:
%%oc -d $node_labels -l 40

MATCH p = (a:Artist)-[:actor|actress*]-(bacon:Artist)
WHERE a.name = 'Keegan-Michael Key' AND bacon.name = 'Kevin Bacon'
RETURN p
LIMIT 10

Tracing through the visualization on the Graph tab, we see that the smallest Bacon number for Keegan-Michael Key is 3.

Variable-length paths give us a shorthand notation (through the `*` character) to specify the length of a sequence of nodes and relationships.

For example, `(a)-[*2]->(b)` is the same as `(a)-[]->()-[]->(b)`. 

Not only are they great for simplifying our statements, but they are also handy if we're interested in a range of lengths. For example, `(a)-[*1..2]->(b)` finds patterns that match `(a)-[]->(b)` or `(a)-[]->()-[]->(b)`.

In our Bacon number query above, we use the pattern `()-[*]-()`, which matches patterns of any path length >= 1, ignoring relationship direction.

To see more examples of variable-length pattern matching, consult the following documentation:
- https://s3.amazonaws.com/artifacts.opencypher.org/openCypher9.pdf, page 11

### 05.03 - Running average

openCypher also provides a few mathematic steps. This includes things like `avg()`, `max()`, `min()`, and mathematical operators like `+`, `-`, `*`, `/`.

Consult the following documentation:
 - `avg()` - https://s3.amazonaws.com/artifacts.opencypher.org/openCypher9.pdf, page 151
 - `max()` - https://s3.amazonaws.com/artifacts.opencypher.org/openCypher9.pdf, page 155
 - `min()` - https://s3.amazonaws.com/artifacts.opencypher.org/openCypher9.pdf, page 156
 
Let's try using the `avg()` step to take a running average of ratings for a particular Artist or production.  In the following two examples we look at the average rating "The Original Kings of Comedy".  In addition, we have a query looking at the average ratings of all movies directed by Roland Emmerich.

In [ ]:
%%oc

MATCH (m:movie {title: 'The Original Kings of Comedy'})<-[e:rated]-()
RETURN avg(e.rating)

In [ ]:
%%oc

MATCH (a:Artist {name: 'Roland Emmerich'})<-[:director]-(m)<-[e:rated]-()
RETURN avg(e.rating)

Yikes!  People must really not like Roland Emmerich's films.

### 05.04 - Another find pattern example

Let's find the most popular genre of movies that were directed by Martin Scorsese and where Leonardo DiCaprio acted in them.


In [ ]:
%%oc

MATCH (actor:Artist {name: 'Leonardo DiCaprio'})<-[:actor]-(movies),
      (movies)-[:director]->(martin {name: 'Martin Scorsese'}),
      (movies)-[:genre]->(g)
RETURN DISTINCT g.genre, count(*)
ORDER BY count(*) DESC

### 05.05 - Using maps

We can construct literal maps in openCypher, by simply specifying the desired keys and corresponding values within curly brackets.

Consult the following documentation:
 - Maps - https://s3.amazonaws.com/artifacts.opencypher.org/openCypher9.pdf, page 20

Here's a good example. List the top 10 movies which have received highest average rating by the people. 

In [ ]:
%%oc

MATCH (a:Artist {name: 'Carrie Fisher'})<-[:actress]-(m)
WHERE EXISTS(m.averageRating)
RETURN {movie: m.title, numratings: m.averageRating}

### 05.06 - Social Rankings

Lastly, let's look at another example for proving connection based ranking.  In the following example, we will look at users and determine which of them are the most highly connected in our graph.


In [ ]:
%%oc

MATCH (p:Person)-[e:knows]->()
WITH DISTINCT p, count(*) AS knows
ORDER BY knows DESC
LIMIT 10
RETURN {firstName: p.firstName, lastName: p.lastName, numknows: knows}

### Review

In the previous notebook we looked at more advanced patterns for using openCypher patterns to find interesting insights within our data.  These are just some of the different patterns that make graph useful.  We will use some of these patterns in later exercises in this workshop.

### End of Section 05 - Move onto the next notebook.